In [ ]:
%pip install optuna
%pip install catboost
%pip install shap

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from imblearn.combine import SMOTEENN
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
import shap
import matplotlib.pyplot as plt
import optuna

from sklearn.model_selection import StratifiedKFold

In [ ]:
# 1. 데이터 로드
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

# train 데이터 복사하여 사용
train_df = train.copy()
test_df = test.copy()


X = train.drop(columns=['임신 성공 여부'], errors='ignore')
y = train['임신 성공 여부']

In [ ]:
# 2. 유니크 값이 1개인 컬럼 삭제
drop_cols = ['불임 원인 - 여성 요인', '난자 채취 경과일']
X.drop(columns=drop_cols, inplace=True, errors='ignore')
test_df.drop(columns=drop_cols, inplace=True, errors='ignore')

# 3. '배란 자극 여부'에 따른 '배란 유도 유형' 수정
if '배란 유도 유형' in X.columns:
    X.loc[(X['배란 자극 여부'] == 0) & (X['배란 유도 유형'] == '알 수 없음'), '배란 유도 유형'] = '시행 없음'
    X.loc[(X['배란 자극 여부'] == 1) & (X['배란 유도 유형'] == '알 수 없음'), '배란 유도 유형'] = '기록되지 않은 시행'

if '배란 유도 유형' in test_df.columns:
    test_df.loc[(test_df['배란 자극 여부'] == 0) & (test_df['배란 유도 유형'] == '알 수 없음'), '배란 유도 유형'] = '시행 없음'
    test_df.loc[(test_df['배란 자극 여부'] == 1) & (test_df['배란 유도 유형'] == '알 수 없음'), '배란 유도 유형'] = '기록되지 않은 시행'

# 4. 숫자형 컬럼 값 정수 변환
cols_to_slice = ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수',
                 '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']
for col in cols_to_slice:
    X[col] = X[col].astype(str).str.extract(r'(\d+)').astype(float).fillna(0).astype(int)
    test_df[col] = test_df[col].astype(str).str.extract(r'(\d+)').astype(float).fillna(0).astype(int)

# 5. 특정 NaN 값 채우기 (가장 많이 등장한 값 적용)
columns_to_fill_most_frequent = ['특정 시술 유형', '배아 생성 주요 이유']
imputer_cat = SimpleImputer(strategy="most_frequent")
X[columns_to_fill_most_frequent] = imputer_cat.fit_transform(X[columns_to_fill_most_frequent])
test_df[columns_to_fill_most_frequent] = imputer_cat.transform(test_df[columns_to_fill_most_frequent])

# 6. 특정 값이 NaN이면 0으로 변환
columns_to_fill_zero = ['단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '이식된 배아 수']
for col in columns_to_fill_zero:
    X[col] = X[col].fillna(0)
    test_df[col] = test_df[col].fillna(0)

In [ ]:
# 2. 범주형 변수 및 수치형 변수 구분
categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = X[col].fillna('Unknown').astype(str)
    test[col] = test[col].fillna('Unknown').astype(str)
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le
    test[col] = test[col].apply(lambda x: x if x in le.classes_ else 'Unknown')
    if 'Unknown' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'Unknown')
    test[col] = le.transform(test[col])

In [ ]:
# 5. 수치형 변수 결측치 처리 (평균값 대체) - 존재하는 변수만 처리
for col in numeric_columns:
    if col in X.columns:
        mean_value = X[col].mean()
        X[col] = X[col].fillna(mean_value)
        if col in test.columns:
            test[col] = test[col].fillna(mean_value)

In [ ]:
def create_features(df):
    """
    모델 성능 향상을 위한 주요 파생 변수 생성
    """
    # 시술 관련 파생 변수
    df["시술 횟수 대비 배아 생성률"] = df["총 생성 배아 수"] / df["총 시술 횟수"].astype(float)
    df["시술 횟수 대비 임신 성공률"] = df["총 임신 횟수"] / df["총 시술 횟수"].astype(float)
    df["시술 횟수 대비 출산 성공률"] = df["총 출산 횟수"] / df["총 시술 횟수"].astype(float)
    df["배아 이식 후 성공률"] = df["총 출산 횟수"] / df["이식된 배아 수"]

    # 배아 및 난자 관련 파생 변수
    df["난자 채취 대비 배아 생성률"] = df["총 생성 배아 수"] / df["수집된 신선 난자 수"]
    df["미세주입 대비 배아 생성률"] = df["미세주입에서 생성된 배아 수"] / df["미세주입된 난자 수"]
    df["배아 해동 후 이식률"] = df["배아 이식 경과일"] / df["배아 해동 경과일"]

    # 불임 원인 개수
    infertility_cols = ["남성 주 불임 원인", "여성 주 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환",
                        "불임 원인 - 남성 요인", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제",
                        "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
                        "불임 원인 - 정자 형태"]
    df["불임 원인 개수"] = df[infertility_cols].sum(axis=1)

    # 시술 당시 나이대 그룹
    df["시술 당시 나이"] = df["시술 당시 나이"].astype(int)
    df["나이대 그룹"] = pd.cut(df["시술 당시 나이"], bins=[20, 30, 40, 50], labels=["20대", "30대", "40대 이상"])

    # 결측치 처리
    df = df.fillna(-1)

    return df

In [ ]:
from sklearn.model_selection import train_test_split

# 파생변수 생성 후 적용된 데이터에서 타겟 변수 분리
X_train_final = train_df.drop(columns=['임신 성공 여부'])
y_train_final = train_df['임신 성공 여부']

# 데이터 분할 (Stratify 옵션 적용)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_final, y_train_final, test_size=0.2, random_state=42, stratify=y_train_final
)

print(f"Train 데이터 크기: {X_train.shape}, Validation 데이터 크기: {X_valid.shape}")

Train 데이터 크기: (205080, 67), Validation 데이터 크기: (51271, 67)


In [ ]:

# 범주형 변수 변환 (Label Encoding) & 새로운 값 처리
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    X_train[col] = X_train[col].fillna('Unknown').astype(str)
    X_valid[col] = X_valid[col].fillna('Unknown').astype(str)

    X_train[col] = le.fit_transform(X_train[col])  # 학습 데이터 변환
    label_encoders[col] = le  # 인코더 저장

    # 검증 데이터 변환 (새로운 값이 나오면 'Unknown'으로 처리)
    X_valid[col] = X_valid[col].apply(lambda x: x if x in le.classes_ else 'Unknown')
    if 'Unknown' not in le.classes_:
        le.classes_ = np.append(le.classes_, 'Unknown')
    X_valid[col] = le.transform(X_valid[col])

# Optuna + CatBoost 튜닝 실행
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 1000, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 4, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 20),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'scale_pos_weight': len(y_train) / sum(y_train),  # 클래스 불균형 자동 조정
        'eval_metric': 'AUC',
        'random_seed': 42,
        'verbose': 0
    }

    # K-Fold Cross Validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []

    for train_idx, valid_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

        model = CatBoostClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=0)

        preds = model.predict_proba(X_val)[:, 1]
        auc_scores.append(roc_auc_score(y_val, preds))

    return np.mean(auc_scores)




In [ ]:
# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 최적의 파라미터 출력
print("Best parameters:", study.best_params)  #Trial 33번이 젤 높음

[I 2025-02-24 19:50:49,395] A new study created in memory with name: no-name-09939a51-ecb8-46a0-b5f0-6e78104cd936
[I 2025-02-24 19:52:58,913] Trial 0 finished with value: 0.7395721678969408 and parameters: {'iterations': 1073, 'learning_rate': 0.10318164117101428, 'depth': 5, 'l2_leaf_reg': 12.72501394354848, 'border_count': 147, 'bagging_temperature': 0.7777738685150848, 'random_strength': 7.914935522421982}. Best is trial 0 with value: 0.7395721678969408.
[I 2025-02-24 19:55:27,704] Trial 1 finished with value: 0.7394767086529532 and parameters: {'iterations': 1929, 'learning_rate': 0.06616376150042003, 'depth': 8, 'l2_leaf_reg': 3.3512217626233247, 'border_count': 184, 'bagging_temperature': 0.4505280569295319, 'random_strength': 8.594522039789808}. Best is trial 0 with value: 0.7395721678969408.
[I 2025-02-24 19:57:42,619] Trial 2 finished with value: 0.7381356639246041 and parameters: {'iterations': 2109, 'learning_rate': 0.17564736064627168, 'depth': 10, 'l2_leaf_reg': 18.4938909

Best parameters: {'iterations': 1128, 'learning_rate': 0.01974308829097581, 'depth': 7, 'l2_leaf_reg': 14.254967637674017, 'border_count': 146, 'bagging_temperature': 0.6490942636012634, 'random_strength': 1.157639810266331}
